In [1]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [2]:
#I use this because my GPU is not working too well with tensorflow
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
import utils
#Utils has both feature computations and data handling

In this notebook I tested my entropy computations as a feature for a dense neural network and compared it to the results without the entropy feature. I also compared it to a convolutional neural network on the raw pixel data.

The Adam optimizer seems to give better performance than RMSProp or SGD and with it the convolutional model converged to 67% accuracy, the dense network with only HOG and hue features converged to 55% accuracy and with my entropy data it converged to 59% accuracy. The convolutional neural network is also deeper, had a much larger input space ($3072$ vs $384$), and took about 10 times longer to train so I would expect it to perform better.

In [64]:
def featurize(data):
    #Computes the HOG, cell hue histograms, and cell entropy histograms
    fn = lambda x: utils.features(x, n_hog_bins = 8, n_hue_bins=8, n_entropy_bins=8, cell_size=(4,4))
    return np.array(fn(img) for img in data)

def train_cifar_model(model, data_tuple):
    x_train, y_train, x_test, y_test = data_tuple
    history = model.fit(
        x_train,
        y_train,
        batch_size=64,
        epochs=10,
        validation_data=(x_test, y_test),
    )
    return history

In [59]:
def make_dense_model():
    model = keras.Sequential(
        [
            layers.Dense(384, activation="relu", input_shape=(384,)),
            layers.Dense(192, activation="relu"),
            layers.Dense(192, activation="relu"),
            layers.Dense(10, activation="softmax")
        ]
    )
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )
    return model

def make_conv_model():
    model = keras.Sequential(
        [
            layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
            layers.MaxPooling2D(2,2),
            layers.Conv2D(64, (3, 3), activation="relu", input_shape=(32, 32, 3)),
            layers.MaxPooling2D(2,2),
            layers.Conv2D(64, (3, 3), activation="relu", input_shape=(32, 32, 3)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dense(10, activation="softmax")
        ]
    )
    model.compile(optimizer=keras.optimizers.Adam(),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
    return model

In [65]:
dense_model = make_dense_model()
dense_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 384)               147840    
_________________________________________________________________
dense_53 (Dense)             (None, 192)               73920     
_________________________________________________________________
dense_54 (Dense)             (None, 192)               37056     
_________________________________________________________________
dense_55 (Dense)             (None, 10)                1930      
Total params: 260,746
Trainable params: 260,746
Non-trainable params: 0
_________________________________________________________________


In [66]:
train_cifar_model(dense_model, utils.load_cifar(raw_data=False))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 3s 59us/sample - loss: 1.5102 - accuracy: 0.4474 - val_loss: 1.4120 - val_accuracy: 0.4931
Epoch 2/10
50000/50000 [==============================] - 3s 55us/sample - loss: 1.2814 - accuracy: 0.5361 - val_loss: 1.2559 - val_accuracy: 0.5484
Epoch 3/10
50000/50000 [==============================] - 3s 61us/sample - loss: 1.1947 - accuracy: 0.5730 - val_loss: 1.2231 - val_accuracy: 0.5636
Epoch 4/10
50000/50000 [==============================] - 3s 68us/sample - loss: 1.1437 - accuracy: 0.5893 - val_loss: 1.2109 - val_accuracy: 0.5752
Epoch 5/10
50000/50000 [==============================] - 3s 62us/sample - loss: 1.0899 - accuracy: 0.6111 - val_loss: 1.2028 - val_accuracy: 0.5798
Epoch 6/10
50000/50000 [==============================] - 3s 56us/sample - loss: 1.0459 - accuracy: 0.6281 - val_loss: 1.2231 - val_accuracy: 0.5716
Epoch 7/10
50000/50000 [==============================] 

In [67]:
conv_model = make_conv_model()
conv_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_6 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_56 (Dense)             (None, 64)              

In [68]:
train_cifar_model(conv_model, utils.load_cifar())

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 38s 766us/sample - loss: 1.8884 - accuracy: 0.3541 - val_loss: 1.4354 - val_accuracy: 0.4657
Epoch 2/10
50000/50000 [==============================] - 34s 674us/sample - loss: 1.3280 - accuracy: 0.5239 - val_loss: 1.2788 - val_accuracy: 0.5511
Epoch 3/10
50000/50000 [==============================] - 33s 668us/sample - loss: 1.1555 - accuracy: 0.5919 - val_loss: 1.1340 - val_accuracy: 0.5987
Epoch 4/10
50000/50000 [==============================] - 35s 699us/sample - loss: 1.0473 - accuracy: 0.6336 - val_loss: 1.0908 - val_accuracy: 0.6214
Epoch 5/10
50000/50000 [==============================] - 32s 637us/sample - loss: 0.9672 - accuracy: 0.6627 - val_loss: 1.0741 - val_accuracy: 0.6300
Epoch 6/10
50000/50000 [==============================] - 31s 628us/sample - loss: 0.9071 - accuracy: 0.6832 - val_loss: 1.0201 - val_accuracy: 0.6504
Epoch 7/10
50000/50000 [====================